In [1]:
%%file myConv.py

import torch
import numpy as np
def myConv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
  def Svertca(matrix):

    #groups exeption
    if (in_channels%groups != 0) or (out_channels%groups!=0):
      raise Exception('in_channels and out_channels must be divisible by groups')

    #генерация bias
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    #padding_mode
    if (padding_mode == 'zeros'):
      pad = torch.nn.ZeroPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'reflect'):
      pad = torch.nn.ReflectionPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'replicate'):
      pad = torch.nn.ReplicationPad2d(2)
      matrix = pad(matrix)
    if (padding_mode == 'circular'):
      pad = torch.nn.CircularPad2d(padding)
      matrix = pad(matrix)

    #генерация ядра
    if type(kernel_size) == tuple:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size[0], kernel_size[1])
    if type(kernel_size) == int:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size, kernel_size)

    #filter = b
    # bias_val = bi

    spisok = []
    for l in range(out_channels):
      feature_map = np.array([]) #генерация пустой feature-map
      for i in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride): #(filter.size - 1)*dilation + 1 при delation
        for j in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride):
          summa = 0
          for c in range (in_channels//groups): #groups
            if groups>1:
              val = matrix[l*(in_channels//groups)+c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*dilation+1:dilation]
            else:
              val = matrix[c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*dilation+1:dilation]
            mini_sum = (val*filter[l][c]).sum()
            summa = summa + mini_sum
          feature_map = np.append(feature_map, float(summa + bias_val[l])) #bias

      spisok.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1, (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1))

    return np.array(spisok), np.array(filter), np.array(bias_val)

  return Svertca

Overwriting myConv.py


In [2]:
%%file conftest.py

from pytest import fixture
import requests

@fixture
def session():
  s = requests.Session()
  s.headers.update({"User-Agent": "pytest requests"})
  s.verify = True
  yield s
  s.close()

Overwriting conftest.py


In [17]:
%%file test_http.py

import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor1 = torch.rand(4, 5, 6)
tensor2 = torch.rand(3, 28, 28)
tensor3 = torch.rand(1, 1, 1)


def test_1(session):
  myFunction = myConv2d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
  result, kernel, bias_val = myFunction(tensor1)
  torchFunction = torch.nn.Conv2d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
  torchFunction.weight.data = torch.tensor(kernel)
  torchFunction.bias.data = torch.tensor(bias_val)
  myResult = str(np.round(result, 2))
  torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))
  assert torchResult == myResult


def test_2(session):
  myFunction2 = myConv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
  result2, kernel2, bias_val2 = myFunction2(tensor2)
  torchFunction2 = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
  torchFunction2.weight.data = torch.tensor(kernel2)
  torchFunction2.bias.data = torch.tensor(bias_val2)
  myResult = str(np.round(result2, 2))
  torchResult = str(np.round(np.array(torchFunction2(tensor2).data), 2))
  assert torchResult == myResult


def test_3(session):
  myFunction3 = myConv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
  result3, kernel3, bias_val3 = myFunction3(tensor3)
  torchFunction3 = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
  torchFunction3.weight.data = torch.tensor(kernel3)
  torchFunction3.bias.data = torch.tensor(bias_val3)
  myResult = str(np.round(result3, 2))
  torchResult = str(np.round(np.array(torchFunction3(tensor3).data), 2))
  assert torchResult == myResult



Overwriting test_http.py


In [18]:
!pytest

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.2, pluggy-1.3.0
rootdir: /content
plugins: anyio-3.7.1
collected 3 items                                                                                  

test_http.py ...                                                                             [100%]

======================================== 3 passed in 2.19s =========================================


In [ ]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor1 = torch.rand(8, 5, 6)

myFunction = myConv2d(in_channels=8, out_channels=4, kernel_size=3, stride=1, padding=0, dilation=1, groups=4, bias=True, padding_mode='zeros')
result, kernel, bias_val = myFunction(tensor1)
torchFunction = torch.nn.Conv2d(in_channels=8, out_channels=4, kernel_size=3, stride=1, padding=0, dilation=1, groups=4, bias=True, padding_mode='zeros')
torchFunction.weight.data = torch.tensor(kernel)
torchFunction.bias.data = torch.tensor(bias_val)

print(result)
print('===================================')
print(np.array(torchFunction(tensor1).data))

[[[3.34844255 3.95108986 4.54495287 4.99099159]
  [3.47761631 4.52002954 5.16909838 5.16851377]
  [3.82740164 4.9305582  4.66527796 5.91074657]]

 [[3.72950912 3.0689764  3.0299809  3.11834979]
  [3.42131066 3.28885055 2.81432533 3.67164302]
  [4.02423239 4.10868788 3.77645254 3.73318553]]

 [[4.89313793 6.34338093 5.44284248 4.74965191]
  [5.9346981  5.67188835 5.82754135 5.80281639]
  [5.5212698  5.21251488 4.69557285 4.83321714]]

 [[4.32730579 4.0435605  3.25495052 3.11661029]
  [4.73951387 4.09995651 4.08717823 4.14778709]
  [5.31597471 4.77475882 5.07710695 5.47960043]]]
[[[3.3484426 3.9510894 4.544953  4.9909916]
  [3.4776165 4.520029  5.1690984 5.168514 ]
  [3.8274016 4.9305577 4.6652775 5.9107456]]

 [[3.7295089 3.0689764 3.029981  3.1183493]
  [3.4213104 3.2888508 2.8143256 3.671643 ]
  [4.024233  4.1086884 3.7764528 3.7331855]]

 [[4.893138  6.34338   5.4428425 4.749652 ]
  [5.934698  5.6718884 5.8275414 5.8028164]
  [5.52127   5.212515  4.695573  4.8332176]]

 [[4.327306  4

In [ ]:
import numpy as np
import torch
import sys
import os
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor2 = torch.rand(3, 10, 10)

myFunction2 = myConv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
result2, kernel2, bias_val2 = myFunction2(tensor2)
torchFunction2 = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
torchFunction2.weight.data = torch.tensor(kernel2)
torchFunction2.bias.data = torch.tensor(bias_val2)

print(result2)
print('===================================')
print(np.array(torchFunction2(tensor2).data))

[[[ 8.65114594  6.63290262  8.87612343  7.70555401  9.6898632
    8.4545269 ]
  [ 7.80007124  7.32617331  8.38645172  8.6049118   7.9856801
    6.71231604]
  [ 6.50906467  6.34039545  8.4602375   7.52484369 10.23536491
    7.70941877]
  [ 6.77037954  8.51693916  6.57404184  7.97103643  6.60268545
    7.91831541]
  [ 6.49199677  5.53396177  8.35616493  6.08350801  8.08309555
    8.14698219]
  [ 5.99466372  6.36886835  6.4908061   7.67802572  6.51692915
    6.84761524]]]
[[[ 8.651146   6.6329036  8.876123   7.7055535  9.689863   8.454527 ]
  [ 7.8000717  7.3261733  8.386452   8.604913   7.98568    6.712316 ]
  [ 6.5090647  6.340394   8.460237   7.5248446 10.235366   7.709418 ]
  [ 6.7703795  8.51694    6.574043   7.971037   6.602685   7.918315 ]
  [ 6.491997   5.533962   8.356165   6.083507   8.0830965  8.146982 ]
  [ 5.9946647  6.368868   6.4908056  7.6780243  6.516929   6.8476157]]]


In [ ]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor3 = torch.rand(1, 1, 1)

myFunction3 = myConv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
result3, kernel3, bias_val3 = myFunction3(tensor3)
torchFunction3 = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
torchFunction3.weight.data = torch.tensor(kernel3)
torchFunction3.bias.data = torch.tensor(bias_val3)

print(result3)
print('===================================')
print(np.array(torchFunction3(tensor3).data))

[[[0.67961389]]]
[[[0.6796139]]]


In [ ]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor4 = torch.rand(4, 5, 5)

myFunction4 = myConv2d(in_channels=4, out_channels=2, kernel_size=3, stride=2, padding=2, dilation=1, groups=2, bias=True, padding_mode='zeros')
result4, kernel4, bias_val4 = myFunction4(tensor4)
torchFunction4 = torch.nn.Conv2d(in_channels=4, out_channels=2, kernel_size=3, stride=2, padding=2, dilation=1, groups=2, bias=True, padding_mode='zeros')
torchFunction4.weight.data = torch.tensor(kernel4)
torchFunction4.bias.data = torch.tensor(bias_val4)

print(result4)
print('===================================')
print(np.array(torchFunction4(tensor4).data))

[[[1.23561835 1.58305216 2.35816383 1.28849316]
  [2.59952831 5.50430155 6.40189981 2.71030331]
  [1.73200881 5.05753469 4.45660257 1.93866241]
  [1.32079434 2.92449093 1.73164964 1.15448403]]

 [[0.39932436 2.71708417 2.60664701 1.11812103]
  [1.62690139 6.49363995 4.50790977 2.38324142]
  [1.24424589 4.76778507 6.88690186 3.02811503]
  [1.06748772 2.47526622 3.49918818 1.57792783]]]
[[[1.2356184  1.5830522  2.3581638  1.2884932 ]
  [2.5995278  5.5043015  6.4019     2.7103033 ]
  [1.7320088  5.057534   4.4566026  1.9386624 ]
  [1.3207943  2.924491   1.7316496  1.154484  ]]

 [[0.39932436 2.717084   2.606647   1.1181211 ]
  [1.6269013  6.49364    4.50791    2.3832412 ]
  [1.2442459  4.767785   6.886902   3.028115  ]
  [1.0674877  2.4752662  3.4991882  1.5779278 ]]]
